In [1]:
#Generate root password
import random, string
password ="olakase" #set password 

#''.join(random.choice(string.ascii_letters + string.digits) for i in range(20))

#Download ngrok
! wget -q -c -nc https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
! unzip -qq -n ngrok-stable-linux-amd64.zip
#Setup sshd
! apt-get install -qq -o=Dpkg::Use-Pty=0 openssh-server pwgen > /dev/null
#Set root password
! echo root:$password | chpasswd
! mkdir -p /var/run/sshd
! echo "PermitRootLogin yes" >> /etc/ssh/sshd_config
! echo "PasswordAuthentication yes" >> /etc/ssh/sshd_config
! echo "LD_LIBRARY_PATH=/usr/lib64-nvidia" >> /root/.bashrc
! echo "export LD_LIBRARY_PATH" >> /root/.bashrc

#Run sshd
get_ipython().system_raw('/usr/sbin/sshd -D &')

#Ask token
print("Copy authtoken from https://dashboard.ngrok.com/auth")
import getpass
authtoken = getpass.getpass()

#Create tunnel
get_ipython().system_raw('./ngrok authtoken $authtoken && ./ngrok tcp 22 &')
#Print root password
print("Root password: {}".format(password))
#Get public address
! curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

Copy authtoken from https://dashboard.ngrok.com/auth
··········
Root password: olakase
tcp://0.tcp.ngrok.io:17322


In [2]:
# if you wanna mount your own google drive
from google.colab import drive
drive.mount('/content/drive') 

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [3]:
#Installing software dependencies, wav2letter backend and finally download a custom version of Librespeech-clean corpus train/validation/test sets

%cd /root
!pip install gdown
!gdown https://drive.google.com/uc?id=1CTXJeQN9Jnj0wx3TJuFjEo2wyoLj1VKL
!tar zxf software3.tar.gz
!chmod 755 env.sh
!./env.sh

/root
Downloading...
From: https://drive.google.com/uc?id=1CTXJeQN9Jnj0wx3TJuFjEo2wyoLj1VKL
To: /root/software3.tar.gz
336MB [00:02, 164MB/s]
Get:1 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease [21.3 kB]
Get:2 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:3 http://archive.ubuntu.com/ubuntu bionic InRelease [242 kB]
Get:4 http://ppa.launchpad.net/marutter/c2d4u3.5/ubuntu bionic InRelease [15.4 kB]
Get:5 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:6 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Get:7 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran35/ InRelease [3,609 B]
Get:8 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic/main amd64 Packages [31.0 kB]
Get:9 http://security.ubuntu.com/ubuntu bionic-security/main amd64 Packages [377 kB]
Get:10 http://security.ubuntu.com/ubuntu bionic-security/restricted amd64 Packages [5,436 B]
Get:11 http://security.ubuntu.com/ubun

In [5]:
#Generate input data format for audio and transcripts files
#From previous last step cell is supossed we downloaded Librispeech-clean (/root/w2l-10.tar.gz a custom short version 
#that ensures 10 minutes per epoch/training just for this tutorial) 
#for original data one (around 28K 10-15s training files) follow instructions in Facebook AI tutorial at wave2letter tutorial for corpus downloading)

#Setting Librispeech data folder environment variable (waveform files and transcripts)
%env W2LDIR=/root/w2l/ 

#Formatting audio and transcripts files to wav2letter format
!python3 wav2letter/tutorials/1-librispeech_clean/prepare_data.py --src $W2LDIR/LibriSpeech --dst $W2LDIR

#Generate lm (downloading 3-gram model, we could also train it with provided kenLM library, 
#included in wav2letter repo. It has been installed in this Google server during previous cell step execution)
#Currently wav2letter repository does not include an integration with, e.g., LSTM language models
#but it will be added in next weeks, allowing training/decoding for completely end-to-end models as authors claim
!python3 wav2letter/tutorials/1-librispeech_clean/prepare_lm.py --dst $W2LDIR

env: W2LDIR=/root/w2l/
analyzing /root/w2l//LibriSpeech/train-clean-10...
writing to /root/w2l/data/train-clean-10...
analyzing /root/w2l//LibriSpeech/dev-clean...
writing to /root/w2l/data/dev-clean...
analyzing /root/w2l//LibriSpeech/test-clean...
writing to /root/w2l/data/test-clean...
creating tokens file /root/w2l/data/tokens.txt...
Done !


--2019-04-06 14:19:47--  http://www.openslr.org/resources/11/3-gram.arpa.gz
Resolving www.openslr.org (www.openslr.org)... 46.101.158.64
Connecting to www.openslr.org (www.openslr.org)|46.101.158.64|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 759636181 (724M) [application/x-gzip]
Saving to: ‘STDOUT’

-                   100%[===================>] 724.45M  27.3MB/s    in 26s     

2019-04-06 14:20:14 (27.5 MB/s) - written to stdout [759636181/759636181]


Converting to lowercase ...


Writing Lexicon file - /root/w2l/lm/lexicon.txt...

Done !


In [6]:
#Training a end-to-end model (CTC) for small subset of Librispeech-clean (10K files around 10-15s) dataset
#Audio is finally decoded with a 3-gram lamguage model (not completely end-to-end)
#--flagsfile /root/wav2letter/tutorials/1-librispeech_clean/train.cfg 
#Set rundir folder, now set to /root/run/ in local server, to your gdrive space
#e.g., --rundir=/content/drive/My\ drive/run so you could keep models and results
#even in the case the server is shutting down


#Original config file sets training iterations/epochs to 100, let's comment that line in config file
#and set it to 5 (around 30 minutes total training) by argument in the following two lines
#note that file config flags are prioritary over argument passed ones

! sed -i s/'--iter=100'/'#--iter=100'/g /root/wav2letter/tutorials/1-librispeech_clean/train.cfg

## Training
#  set --iter=25 epochs (around 150 minutes training) to get same results than in slides ones
!chmod +x /root/wav2letter/build/Train
!/root/wav2letter/build/Train train -nthread 1 --criterion=ctc --flagsfile /root/wav2letter/tutorials/1-librispeech_clean/train.cfg --iter 5 --itersave --logtostderr

I0406 14:23:42.677392 39662 Train.cpp:57] Reading flags from file /root/wav2letter/tutorials/1-librispeech_clean/train.cfg
I0406 14:23:42.765573 39662 Train.cpp:133] Gflags after parsing 
--flagfile=; --fromenv=; --tryfromenv=; --undefok=; --tab_completion_columns=80; --tab_completion_word=; --help=false; --helpfull=false; --helpmatch=; --helpon=; --helppackage=false; --helpshort=false; --helpxml=false; --version=false; --adambeta1=0.90000000000000002; --adambeta2=0.999; --am=; --arch=network.arch; --archdir=/root/wav2letter/tutorials/1-librispeech_clean; --attention=content; --attnWindow=no; --batchsize=4; --beamscore=25; --beamsize=2500; --channels=1; --criterion=ctc; --critoptim=sgd; --datadir=/root/w2l/; --dataorder=input; --devwin=0; --emission_dir=; --enable_distributed=false; --encoderdim=0; --eostoken=false; --everstoredb=false; --fftcachesize=1; --filterbanks=40; --flagsfile=/root/wav2letter/tutorials/1-librispeech_clean/train.cfg; --forceendsil=false; --gamma=1; --garbage=fal

In [0]:
# Librispeech test-clean just too big for our tutorial so let's make it smaller now with just 10 files for decoding,...
# as you see, that is the reason i prefer being able to login into the server machine :)
! cd /root/w2l/data/ && mkdir /root/w2l/data/test-clean-10 
! cd /root/w2l/data/test-clean-10 && for file in $(ls --color=never /root/w2l/data/test-clean/*flac | head -10) ; do  ln -s $file .; done
! cd /root/w2l/data/test-clean-10 && for file in $(ls --color=never /root/w2l/data/test-clean/*id | head -10) ; do  ln -s $file .; done
! cd /root/w2l/data/test-clean-10 && for file in $(ls --color=never /root/w2l/data/test-clean/*wrd | head -10) ; do  ln -s $file .; done
! cd /root/w2l/data/test-clean-10 && for file in $(ls --color=never /root/w2l/data/test-clean/*tkn | head -10) ; do  ln -s $file .; done
! cd /root/



In [10]:
#Decoding Librispeech test-clean with best trained model selected on the dev-clean data set, that is, 001_model_data#dev-clean.bin
#--flagsfile /root/wav2letter/tutorials/1-librispeech_clean/decode.cfg
! chmod +x /root/wav2letter/build/Decoder
! mkdir -p /root/run/librispeech_clean_decodelogs/
! /root/wav2letter/build/Decoder --lm /root/w2l/lm/3-gram.arpa --am /root/run/librispeech_clean_trainlogs_ctc/001_model_data#dev-clean.bin --sclite /root/run/librispeech_clean_decodelogs/  --lexicon /root/w2l/lm/lexicon.txt --test data/test-clean-10 --lmweight 2.5 --wordscore 1 --beamsize 1000 --beamscore 25 --silweight=-0.5 --nthread 1 --nthread_decoder 1 --smearing max --show  --logtostderr

I0406 17:14:37.410166 44232 Decode.cpp:72] [Network] Reading acoustic model from /root/run/librispeech_clean_trainlogs_ctc/001_model_data#dev-clean.bin
I0406 17:14:38.386291 44232 Decode.cpp:76] [Network] Sequential [input -> (0) -> (1) -> (2) -> (3) -> (4) -> (5) -> (6) -> (7) -> (8) -> (9) -> (10) -> (11) -> (12) -> (13) -> (14) -> (15) -> (16) -> (17) -> (18) -> (19) -> (20) -> output]
	(0): View (-1 1 40 0)
	(1): Conv2D (40->256, 8x1, 2,1, SAME,SAME, 1, 1) (with bias)
	(2): ReLU
	(3): Conv2D (256->256, 8x1, 1,1, SAME,SAME, 1, 1) (with bias)
	(4): ReLU
	(5): Conv2D (256->256, 8x1, 1,1, SAME,SAME, 1, 1) (with bias)
	(6): ReLU
	(7): Conv2D (256->256, 8x1, 1,1, SAME,SAME, 1, 1) (with bias)
	(8): ReLU
	(9): Conv2D (256->256, 8x1, 1,1, SAME,SAME, 1, 1) (with bias)
	(10): ReLU
	(11): Conv2D (256->256, 8x1, 1,1, SAME,SAME, 1, 1) (with bias)
	(12): ReLU
	(13): Conv2D (256->256, 8x1, 1,1, SAME,SAME, 1, 1) (with bias)
	(14): ReLU
	(15): Conv2D (256->256, 8x1, 1,1, SAME,SAME, 1, 1) (with bias)

In [0]:
!cd /root

In [0]:
#If you are working just from shell terminal, better to keep this notebook alive by a dummy command 
# so google do not kick us out due to innactivity
import time
while True:
  time.sleep(60)
  print("hola")

In [0]:
#In case you loose connection, try it again by restarting the ssh tunnel
#If it does not work, you should restart this google collaborative instance
#and start again executing cells from the first one

#Create tunnel
get_ipython().system_raw('./ngrok authtoken $authtoken && ./ngrok tcp 22 &')
#Print root password
print("Root password: {}".format(password))
#Get public address
! curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

Root password: olakase
tcp://0.tcp.ngrok.io:17452
